In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


지문을 읽고 주제 분류하기

다음과 같은 입력 데이터를 데이터프레임으로 저장한다. 입력 데이터는 음식 관련 및 스포츠 관련 지문으로 구성되어 있다.

In [2]:
paragraph_dict_list = [
    {'paragraph': 'Dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
    {'paragraph': 'Srvice can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
    {'paragraph': 'Portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
    {'paragraph': 'We started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
    {'paragraph': 'The biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
    {'paragraph': 'The garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
    {'paragraph': 'Our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
    {'paragraph': 'What i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
    {'paragraph': 'The drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
    {'paragraph': 'Despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    {'paragraph': 'The four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
    {'paragraph': 'The briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
    {'paragraph': 'Stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
    {'paragraph': 'When it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
    {'paragraph': 'The team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
    {'paragraph': 'The perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
    {'paragraph': 'Liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
    {'paragraph': 'Alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
    {'paragraph': 'But the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
    {'paragraph': 'Then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df

,paragraph,category
0,Dishplace is located in sunnyvale downtown the...,food
1,Srvice can be slower during busy hours but our...,food
2,Portions are huge both french toast and their ...,food
3,We started with apps going the chicken and waf...,food
4,The biscuits and gravy was too salty two peopl...,food
5,The garlic fries were a great starter (and a h...,food
6,Our meal was excellent i had the pasta ai form...,food
7,What i enjoy most about palo alto is so many r...,food
8,The drinks came out fairly quickly a good two ...,food
9,Despite the not so good burger the service was...,food


데이터 전처리

LSTM 모델이 입력 데이터를 처리할 수 있도록 수치값으로 변경한다.  
텍스트인 입력값을 수치로 변환하기 위해서 지문에 사용된 모든 단어들을 모아서 중복을 제거한 후 단어 리스트를 만든다.

In [3]:
# 중복을 제거해서 지문에 사용된 단어를 모으기 위해서 빈 set을 선언한다.
results = set()
# print(type(results)) # <class 'set'>
# print(type(df)) # <class 'pandas.core.frame.DataFrame'>
# print(type(df.paragraph)) # <class 'pandas.core.series.Series'>
# print(type(df.paragraph.str)) # <class 'pandas.core.strings.StringMethods'>
# print(type(df.paragraph.str.lower())) # <class 'pandas.core.series.Series'>

# 데이터프레임의 paragraph열(Series)을 모두 소문자로 변환한 후 공백을 경계로 나눠서 set에 저장한다.
# results = df.paragraph.str.lower().str.split()
# print(type(results))
df.paragraph.str.lower().str.split().apply(results.update)
print(len(results))
print(results)

537
{'roma', 'haven', 'surprised', 'soft', 'served', 'team', 'bit', 'retrieving', 'however', 'bottom', 'soon', 'odds', 'can', 'san', 'she', 'more', 'without', 'win', 'confidence', 'one', 'for', 'williams', 'murray', 'set', 'champions', 'portions', 'no83', 'liverpool', 'parking', 'salty', 'croatia', 'know', 'no2', 'victory', 'hungry', 'starter', 'up', 'champion', 'winning', 'had', 'potential', 'net', 'highly', 'generous', 'laughed', 'avoided', 'entree', 'got', 'didnt', 'style', 'quickly', 'waffle', 'hip', 'injury', 'serena', 'pancakes', 'course', 'sweet', 'ai', 'truly', 'oh-so-familiar', 'stephens', 'losses', 'fries', 'are', 'no46', 'sisters', 'courteous', 'apps', 'goalkeeper', 'real', 'looked', 'tasted', 'if', 'spot', 'rankings', 'you', '1', 'losing', 'reigning', 'tournaments', 'via', 'edged', 'wanted', 'busy', 'scored', 'help', 'proved', 'formaggi', 'pessimistic', 'four', 'kerber', 'chasing', 'modric', 'chance', 'quarter—except', 'meal', 'moment', 'exhilarating', 'league', 'patience',

set에 저장된 데이터를 아래와 같은 형태의 딕셔너리로 만든다.  
{0: 'happy', 1: 'defending', 2: 'better', ..., 534: 'ran', 535: 'pulled', 536: 'seeding'}

In [4]:
# enumerate() 함수는 인수로 지정된 객체에 저장된 내용을 (인덱스, 데이터) 형태로 리턴한다.
# for index, word in enumerate(results):
    # print(index, word)
idx2word = dict(enumerate(results))
print(type(idx2word))
print(idx2word)

<class 'dict'>
{0: 'roma', 1: 'haven', 2: 'surprised', 3: 'soft', 4: 'served', 5: 'team', 6: 'bit', 7: 'retrieving', 8: 'however', 9: 'bottom', 10: 'soon', 11: 'odds', 12: 'can', 13: 'san', 14: 'she', 15: 'more', 16: 'without', 17: 'win', 18: 'confidence', 19: 'one', 20: 'for', 21: 'williams', 22: 'murray', 23: 'set', 24: 'champions', 25: 'portions', 26: 'no83', 27: 'liverpool', 28: 'parking', 29: 'salty', 30: 'croatia', 31: 'know', 32: 'no2', 33: 'victory', 34: 'hungry', 35: 'starter', 36: 'up', 37: 'champion', 38: 'winning', 39: 'had', 40: 'potential', 41: 'net', 42: 'highly', 43: 'generous', 44: 'laughed', 45: 'avoided', 46: 'entree', 47: 'got', 48: 'didnt', 49: 'style', 50: 'quickly', 51: 'waffle', 52: 'hip', 53: 'injury', 54: 'serena', 55: 'pancakes', 56: 'course', 57: 'sweet', 58: 'ai', 59: 'truly', 60: 'oh-so-familiar', 61: 'stephens', 62: 'losses', 63: 'fries', 64: 'are', 65: 'no46', 66: 'sisters', 67: 'courteous', 68: 'apps', 69: 'goalkeeper', 70: 'real', 71: 'looked', 72: 'ta

{key: value, ...} 형태의 딕셔너리를 아래와 같이 {value: key, ...} 형태의 딕셔너리로 변환한다.  
{'happy': 0, 'defending': 1,'better': 2, ...,'ran': 534,'pulled': 535,'seeding': 536}

In [5]:
word2idx = {value.replace('(', '').replace(')', ''): key for key, value in idx2word.items()}
print(type(word2idx))
print(word2idx)

<class 'dict'>
{'roma': 0, 'haven': 1, 'surprised': 2, 'soft': 3, 'served': 4, 'team': 5, 'bit': 6, 'retrieving': 7, 'however': 8, 'bottom': 9, 'soon': 10, 'odds': 11, 'can': 12, 'san': 13, 'she': 14, 'more': 15, 'without': 16, 'win': 17, 'confidence': 18, 'one': 19, 'for': 20, 'williams': 21, 'murray': 22, 'set': 23, 'champions': 24, 'portions': 25, 'no83': 26, 'liverpool': 27, 'parking': 28, 'salty': 29, 'croatia': 30, 'know': 31, 'no2': 32, 'victory': 33, 'hungry': 34, 'starter': 35, 'up': 36, 'champion': 37, 'winning': 38, 'had': 39, 'potential': 40, 'net': 41, 'highly': 42, 'generous': 43, 'laughed': 44, 'avoided': 45, 'entree': 46, 'got': 47, 'didnt': 48, 'style': 49, 'quickly': 50, 'waffle': 51, 'hip': 52, 'injury': 53, 'serena': 54, 'pancakes': 55, 'course': 56, 'sweet': 57, 'ai': 58, 'truly': 59, 'oh-so-familiar': 60, 'stephens': 61, 'losses': 62, 'fries': 63, 'are': 64, 'no46': 65, 'sisters': 66, 'courteous': 67, 'apps': 68, 'goalkeeper': 69, 'real': 70, 'looked': 71, 'tasted

In [6]:
print(word2idx['happy'], word2idx['eliminated'], word2idx['pulled'])

141 332 364


word2idx 딕셔너리를 활용해서 모든 지문을 구성하는 단어 목록을 수치로 변환한 열을 데이터프레임에 추가한다.

In [7]:
# 지문을 구성하는 모든 단어를 수치로 변환하는 함수를 만든다.
def encode_paragraph(paragraph):
    words = paragraph.split()
    # print(words)
    encoded = []
    for word in words:
        encoded.append([word2idx[word.lower().replace('(', '').replace(')', '')]])
    # print(encoded)
    return encoded

In [8]:
# 지문이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df.head()

,paragraph,category,enc_paragraph
0,Dishplace is located in sunnyvale downtown the...,food,"[[118], [136], [366], [285], [435], [113], [48..."
1,Srvice can be slower during busy hours but our...,food,"[[520], [12], [437], [469], [522], [84], [474]..."
2,Portions are huge both french toast and their ...,food,"[[25], [64], [479], [527], [450], [246], [326]..."
3,We started with apps going the chicken and waf...,food,"[[407], [212], [345], [68], [360], [260], [408..."
4,The biscuits and gravy was too salty two peopl...,food,"[[260], [397], [326], [273], [385], [143], [29..."


분류 항목(food[1, 0], sports[0, 1])을 원-핫 인코딩으로 수치로 변환한 열을 데이터프레임에 추가한다.  

In [9]:
# 분류 항목을 원-핫 인코딩 수치로 변환하는 함수를 만든다.
def encode_category(category):
    if category == 'food':
        return [1, 0]
    else:
        return [0, 1]

In [10]:
# 분류 항목이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_category'] = df.paragraph.apply(encode_category)
df.head()

,paragraph,category,enc_paragraph,enc_category
0,Dishplace is located in sunnyvale downtown the...,food,"[[118], [136], [366], [285], [435], [113], [48...","[0, 1]"
1,Srvice can be slower during busy hours but our...,food,"[[520], [12], [437], [469], [522], [84], [474]...","[0, 1]"
2,Portions are huge both french toast and their ...,food,"[[25], [64], [479], [527], [450], [246], [326]...","[0, 1]"
3,We started with apps going the chicken and waf...,food,"[[407], [212], [345], [68], [360], [260], [408...","[0, 1]"
4,The biscuits and gravy was too salty two peopl...,food,"[[260], [397], [326], [273], [385], [143], [29...","[0, 1]"


Dynamic RNN은 입력값의 다양한 길이를 고려하여 결과값을 출력하므로 Dynamic RNN이 각 입력값에 실제 길이(단어의 개수)를 알 수 있도록 각 지문별 단어의 개수를 계산해서  
계산된 값을 데이터프레임에 추가한다.

In [11]:
# 지문을 구성하는 단어의 개수를 세는 함수를 만든다.
def word_cnt(paragraph):
    return len(paragraph.split())

In [12]:
# 지문을 구성하는 단어의 개수를 데이터프레임에 추가한다.
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,Dishplace is located in sunnyvale downtown the...,food,"[[118], [136], [366], [285], [435], [113], [48...","[0, 1]",53
1,Srvice can be slower during busy hours but our...,food,"[[520], [12], [437], [469], [522], [84], [474]...","[0, 1]",19
2,Portions are huge both french toast and their ...,food,"[[25], [64], [479], [527], [450], [246], [326]...","[0, 1]",42
3,We started with apps going the chicken and waf...,food,"[[407], [212], [345], [68], [360], [260], [408...","[0, 1]",43
4,The biscuits and gravy was too salty two peopl...,food,"[[260], [397], [326], [273], [385], [143], [29...","[0, 1]",82
5,The garlic fries were a great starter (and a h...,food,"[[260], [464], [63], [448], [352], [492], [35]...","[0, 1]",24
6,Our meal was excellent i had the pasta ai form...,food,"[[507], [96], [385], [375], [500], [39], [260]...","[0, 1]",50
7,What i enjoy most about palo alto is so many r...,food,"[[158], [500], [436], [149], [477], [460], [24...","[0, 1]",43
8,The drinks came out fairly quickly a good two ...,food,"[[260], [319], [415], [137], [125], [50], [352...","[0, 1]",49
9,Despite the not so good burger the service was...,food,"[[194], [260], [190], [487], [197], [271], [26...","[0, 1]",82


텐서플로우의 RNN은 항상 같은 길이의 시퀀스를 받아야 한다.  
길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어 모든 시퀀스의 길이를 동일하게 설정한다.  
패딩이 RNN 계산에 영향을 끼치지 않도록 Dynamic RNN은 패딩 이전의 입력 시퀀스의 실제 길이를 파라미터로 받아 상태값 계산시 패딩을 제외시킨다.

In [13]:
# 최고로 긴 지문의 단어 수를 구한 후 모든 지문의 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 만들기 위해 
# 최고로 긴 지문의 단어 수를 계산한다.
max_word_cnt = 0
#for row in df.paragraph:
    #if len(row.split()) > max_word_cnt:
        #max_word_cnt = len(row.split())
#for seq in df.seq_length:
    #if seq > max_word_cnt:
        #max_word_cnt = seq
        
max_word_cnt = df.seq_length.max()
print(max_word_cnt)

91


In [14]:
# 모든 지문에 패딩을 집어넣어 최고로 긴 문장과 동일한 길이를 갖게하는 함수를 만든다.
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
    return enc_paragraph

In [15]:
# 모든 지문에 패딩이 적용된 결과를 데이터프레임에 적용한다.
df['enc_paragraph'] = df.enc_paragraph.apply(sequence_padding)
print(len(df.enc_paragraph[1]))
print(df.enc_paragraph[1])
print('=' * 80)
print(len(df.enc_paragraph[12]))
print(df.enc_paragraph[12])

91
[[520], [12], [437], [469], [522], [84], [474], [274], [507], [142], [385], [67], [326], [86], [179], [174], [492], [46], [468], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
91
[[61], [514], [494], [499], [398], [111], [53], [285], [484], [49], [307], [17], [494], [177], [461], [466], [497], [306], [530], [523], [26], [14], [308], [318], [87], [158], [352], [153], [373], [423], [14], [136], [38], [260], [325], [515], [81], [90], [119], [461], [24], [456], [476], [260], [501], [175], [260], [450], [234], [398], [108], [279], [186], [301], [315], [14], [342], [307], [260], [501], [285], [446], [226], [523

모델에서 사용할 입력값, 실제값, Dynamic RNN의 sequence_length 파라미터를 만든다.

In [16]:
# 넘파이 array() 메소드는 인수로 파이썬의 리스트나 튜플 형태의 데이터를 받아서 넘파이 배열로 변환하는 기능을 하기 때문에 
# 데이터프레임의 시리즈를 바로 넣어주면 안되고 tolist() 메소드를 실행해서 시리즈를 리스트 형태로 변환한 후 넣어준다.
enc_paragraph = np.array(df.enc_paragraph.tolist())
# print(type(df.enc_paragraph)) # <class 'pandas.core.series.Series'>
# print(type(enc_paragraph.tolist())) # <class 'list'>
# print(type(enc_paragraph)) # <class 'numpy.ndarray'>
enc_category = np.array(df.enc_category.tolist()) # 실제값
#print(enc_category)
sequence_length = np.array(df.seq_length.tolist()) # Dynamic RNN의 sequence_length 파라미터 값
#print(sequence_length)

train_X = enc_paragraph
trian_Y = enc_category
print(train_X.shape, trian_Y.shape)

(20, 91, 1) (20, 2)


텐서플로우로 지문을 읽고 주제를 분류하는 모델을 만든다.  
<img src="./images/LSTM3.png" align="left" width="1000"/> 

문맥 벡터(contextualized vector) 생성 단계  
인덱스를 임베딩으로 변환한다. 임베딩은 학습 과정을 통해 단어 유사도를 포함하게 되어 문맥 벡터를 생성하는데 도움을 준다.  
인간의 언어(자연거)는 수치화되어있지 않은 데이터이기 때문에 머신러닝, 딥러닝 기법을 사로 사용할 수 없다.  
그래서 자연어 처리에서 특징을 추출해서 수치화 해줘야 하는데 이때 사용하는 것이 '언어의 벡터화'이다. 이런 백터화 과정을 word embedding이라고 한다.  
LSTM에 임베딩된 시퀀스를 입력해서 최종 상태값을 출력한다. => 최종 상태값이 문맥 벡터이다.  

주제(food, sports) 분류 단계  
문맥 벤터를 덴즈 레이어에 입력하고 출력값을 노드가 2개인 덴즈 레이어에 입력한 후 노드가 2개인 덴즈 레이어의 출력값을 소프트 맥스에 입력시켜 food, sports에 대한 예측값을 계산한다.

모델을 구현한다.

In [17]:
# 초기화
tf.set_random_seed(35)
learning_rate = 0.001 # 학습율
n_epoch = 300 # 학습 횟수

# 입력값과 출력값을 저장할 플레이스홀더(placeholder)를 만든다.
# 입력값은 문장을 구성하는 단어들의 인덱스이며, 그 길이는 지문을 구성하는 단어의 최대 개수(max_word_cnt)이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, max_word_cnt, 1]) # 입력값을 기억할 placeholder, (20, 91, 1)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # 실제값을 기억할 placeholder, (20, 2)

# 워드 임베딩 레이어, 임베딩 레이어는 입력값(인덱스)를 입력받아 5차원 벡터의 임베딩을 출력한다.
# layers.dense() 메소드는 densely-connect layer 즉, '완전 연결 계층'을 만들어 준다.
embedding = tf.layers.dense(X, 5)
# LSTM 셀은 64차원 벡터의 생성값을 출력한다.
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)

# 출력값과 상태값을 저장한다. 상태값이 문맥 벡터로 사용되며 dense layer의 입력값이 된다.
output, state = tf.nn.dynamic_rnn(cell=cell, dtype=tf.float32, inputs=embedding, sequence_length=sequence_length)

# 주제 분류는 두 개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개 노드를 가지고 있고, 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며 이 2개의 노드가 소프트 맥스의 입력으로 들어간다.
dense_layer = tf.layers.dense(state.h, 32)
# logit은 food, sports를 원-핫 인코딩으로 구분하기 위해 2차원 벡터로 구성한다.
logit = tf.layers.dense(dense_layer, 2)

# 손실 함수 - 크로스 엔트로피
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y)
loss = tf.reduce_mean(cross_entropy)

# Adam 옵티마이저
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


모델 요약

In [23]:
# 문맥 벡터 생성 
# 입력값은 단어들의 인덱스이며 그 길이는 항상 91이다.
print(X) # Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)

# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력한다.
print(embedding) # Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)

# LSTM 셀은 64차원의 상태값을 출력한다.
print(state.h) # Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)

# 문맥 벡터를 사용해 지문의 주제 분류하기, 주제 분류는 2개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개의 노드를 가지고 있다.
print(dense_layer) # Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
# 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며, 이 2개의 노드는 소프트 맥스의 입력값으로 들어간다.
print(logit) # Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


학습시킨다.

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epoch + 1):
        #sess.run(optimizer, feed_dict={X: train_X, Y: trian_Y})
        #train_loss = sess.run(loss, feed_dict={X: train_X, Y: trian_Y})
        _, train_loss = sess.run([optimizer, loss], feed_dict={X: train_X, Y: trian_Y})
        
        if epoch % 50 == 0:
            predict = tf.nn.softmax(logit) # 예측
            # 정확도 계산
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
            cur_acc = accuracy.eval(feed_dict={X: train_X, Y: trian_Y})
            print('epoch: {:3d}, loss: {:6.4f}, acc: {:6.4f}'.format(epoch, train_loss, cur_acc))

epoch:   0, loss: 0.4698, acc: 1.0000
epoch:  50, loss: 0.0002, acc: 1.0000
epoch: 100, loss: 0.0001, acc: 1.0000
epoch: 150, loss: 0.0000, acc: 1.0000
epoch: 200, loss: 0.0000, acc: 1.0000
epoch: 250, loss: 0.0000, acc: 1.0000
epoch: 300, loss: 0.0000, acc: 1.0000
